# Hicks CSTR
(From Derivation of Stirred Tank Reactor Optimal Control by L.T. Biegler)

Consider the liquid-phase stirred tank reactor shown in Figure 1. We assume constant liquid volume ($V$), flow ($F$), density ($\rho$), heat capacity ($C_{p}$), coolant
temperature ($T_{c}$), coolant flow ($F_{c}$), overall heat transfer coefficient ($U$), cross
sectional area ($A_{c}$) and heat of reaction ($H$), as well as a first order Arrhenius
rate law.

### 0. Introduction

![Hicks](./img/hicks.png)

The problem is given by the ODEs:
\begin{align}
\min _{u} \quad & \int_{0}^{t_{f}}  \alpha_{1} \left(c - c(t) \right)^{2} + \alpha_{2} \left( T - T(t) \right)^{2} + \alpha_{3} \left(u - u(t) \right)^{2} \\
\dfrac{dc}{dt} &= \left(1 - c\left(t \right) \right) / \theta - k_{10} \exp \left(-n/T \right) c\left( t \right) , \quad c \left(0 \right) = c_{0} \\
\dfrac{dT}{dt} &= \left(y_{f} - T\left(t \right) \right)/\theta - k_{10} \exp \left( -n / T \right) c (t) + \alpha u \left(y_{c} - T\right), \quad T \left( 0 \right) = T_{0}
\end{align}

### 1. Model

In [1]:
import matplotlib.pyplot as plt
from pyomo.core.expr import exp
from pyomo.core.base.numvalue import value
from pyomo.dae import *
#: pyomo imports
from pyomo.environ import *
from pyomo.opt import SolverFactory, SolverStatus

In [2]:
#: Number of finite elements
nfe = 100
#: Number of collocation points
ncp = 3
m = ConcreteModel()

In [3]:
m.nfe = nfe
m.ncp = ncp

m.t = ContinuousSet(bounds=(0, 1))

### 2. Parameters

In [4]:
alpha_init = {0: 1.95e-04, 1: 1e+06, 2: 2e+03, 3: 1e-03}

In [5]:
#: Scalars
m.cinit = Param(initialize=0.1367)
m.tinit = Param(initialize=0.7293)
m.uinit = Param(initialize=390.0)

m.cdes = Param(initialize=0.0944)
m.tdes = Param(initialize=0.7766)
m.udes = Param(initialize=340)
m.k10 = Param(initialize=300)
m.n = Param(initialize=5)

m.cf = Param(initialize=7.6)
m.tf = Param(initialize=300)
m.tc = Param(initialize=290)

m.theta = Param(initialize=20)
m.yf = Param(initialize=0.3947)
m.yc = Param(initialize=0.3816)
m.time = Param(initialize=10)
m.point = Param(initialize=0)
m.slopec = Param()
m.slopet = Param()
m.slopeu = Param()
m.ii = Param()
m.jj = Param()

m.alpha = Param([0, 1, 2, 3], initialize=alpha_init)

Generate initial guesses values.

In [6]:
point = 0
slopec = (value(m.cdes) - value(m.cinit)) / (m.nfe * m.ncp)
slopet = (value(m.tdes) - value(m.tinit)) / (m.nfe * m.ncp)
slopeu = (value(m.udes) - value(m.uinit)) / (m.nfe * m.ncp)

cguess = {}
tguess = {}
ttguess = {}
uguess = {}

for i in range(1, m.nfe + 1):
    for j in range(1, m.ncp + 1):
        point += 1
        cguess[i, j] = slopec * point + value(m.cinit)
        tguess[i, j] = slopet * point + value(m.tinit)
        ttguess[i, j] = value(m.time) * point
        uguess[i, j] = slopeu * point + value(m.uinit)

### 3. Variables

In [7]:
#: One can pass the dictionary, as long as the keys are defined within the index set.
m.C = Var(m.t, initialize=cguess)
m.T = Var(m.t, initialize=tguess)
m.u = Var(m.t, initialize=uguess)
# m.tt = Var(m.t, initialize=ttguess)

In [8]:
m.dC_dt = DerivativeVar(m.C)
m.dT_dt = DerivativeVar(m.T)

In [9]:
m.phi = Var()

### 4. Constraints

In [10]:
def _odec_rule(mod, t):
    if t > 0:
        return mod.dC_dt[t] == \
               ((1 - mod.C[t]) / mod.theta - mod.k10 * exp(-mod.n / mod.T[t]) * mod.C[t])
    else:
        return Constraint.Skip

In [11]:
def _odet_rule(mod, t):
    if t > 0:
        return mod.dT_dt[t] == \
               ((mod.yf - mod.T[t]) / mod.theta + mod.k10 * exp(-mod.n / mod.T[t]) * mod.C[t] -
                mod.alpha[0] * mod.u[t] * (mod.T[t] - mod.yc))
    else:
        return Constraint.Skip

In [12]:
m.OdeC = Constraint(m.t, rule=_odec_rule)
m.OdeT = Constraint(m.t, rule=_odet_rule)

In [13]:
def _ic_rule(mod):
    return mod.C[0] == mod.cinit

m.IC = Constraint(rule=_ic_rule)

In [14]:
def _it_rule(mod):
    return mod.T[0] == mod.tinit

m.IT = Constraint(rule=_it_rule)

In [15]:
def objective_rule(mod):
    return sum((mod.alpha[1] * (mod.cdes - mod.C[t]) ** 2 +
                mod.alpha[2] * (mod.tdes - mod.T[t]) ** 2 +
                mod.alpha[3] * (mod.udes - mod.u[t]) ** 2)
               for t in m.t if t > 0)


m.fobj = Objective(sense=minimize, rule=objective_rule)

### 5. Transformation and solution

In [16]:
dae = TransformationFactory('dae.collocation')
dae.apply_to(m, nfe=m.nfe, ncp=m.ncp, scheme='LAGRANGE-RADAU')

Set initial values

In [17]:
for var in m.C.itervalues():
    var.setlb(0)
    var.setub(1)

for var in m.T.itervalues():
    var.setlb(0.1)
    var.setub(1)

for var in m.u.itervalues():
    var.setlb(0)
    var.setub(500)

for t in m.t:
    m.C[t].set_value(slopec * t + value(m.cinit))
    m.T[t].set_value(slopet * t + value(m.tinit))
    m.u[t].set_value(slopeu * t + value(m.uinit))

Solve

In [18]:
ipopt = SolverFactory('ipopt')
results = ipopt.solve(m, tee=True)

/tmp/tmpgqkyxrbn.pyomo.nl -AMPL 
Ipopt vanilla0: 
This is Ipopt version vanilla0, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:     5102
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      902

Total number of variables............................:     1502
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      902
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1202
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.7843094e+03 1.58e-04 1.18

Set up some plots.

In [19]:
templ = []
cl = []
tl = []
ul = []
for key in m.t:
    var = m.u[key]
    if var.stale:
        continue
        
    templ.append(value(m.T[key]))
    cl.append(value(m.C[key]))
    tl.append(key)
    ul.append(value(m.u[key]))

In [1]:
ax1 = plt.subplot(3, 1, 1)
plt.plot(tl, templ)

plt.title("Temperature")
plt.subplot(3, 1, 2, sharex=ax1)
plt.plot(tl, cl)

plt.title("Concentration")
plt.subplot(3, 1, 3, sharex=ax1)
plt.plot(tl, ul)
plt.xlabel("time")
plt.title("Control input")
plt.show()

NameError: name 'plt' is not defined

### Credits:
 - David Thierry (Carnegie Mellon University @2019)